In [ ]:
const name = "ItemSimilarity";

In [ ]:
using CSV
using DataFrames
using LinearAlgebra
import NBInclude: @nbinclude
using SparseArrays
@nbinclude("../TrainingAlphas/Alpha.ipynb");

In [ ]:
anime_to_uid = DataFrame(CSV.File("../../data/processed_data/anime_to_uid.csv"));
anime = DataFrame(
    CSV.File("../../data/processed_data/anime.csv", ntasks = 1; stringtype = String),
)
anime_to_uid = innerjoin(anime_to_uid, anime, on = "animeid" => "anime_id");

In [ ]:
Logging.disable_logging(Logging.Warn)
Z = read_params("all/Transformer/norm/checkpoints/6")["m"].embed.embeddings.item.embedding
Z = Z[:, 1:num_items()];

In [ ]:
#cosine distance
normalize!.(eachcol(Z))
W = Z' * Z;

In [ ]:
function get_neighborhood(item, W, K; transpose = false)
    if transpose
        weights = W[item, :]
    else
        weights = W[:, item]
    end
    order = sortperm(weights, rev = true)[1:K]
    order[1:K], weights[order[1:K]]
end;

In [ ]:
function get_series(df)
    df = copy(df)
    df[!, :uid] = df[:, :anime] .- 1
    sim = innerjoin(df, anime_to_uid, on = "uid")
    sort(sim, :weight, rev = true)
end;

In [ ]:
function display(df::DataFrame)
    df = select(df, :title, :weight)
    headers = titlecase.(replace.(names(df), "_" => " "))
    Base.show(
        stdout,
        MIME("text/html"),
        df;
        allow_html_in_cells = true,
        header = headers,
        nosubheader = true,
        row_number_column_title = "Rank",
        top_left_str = "",
    )
end;

In [ ]:
function display_related_series(mal_uid)
    uid = only(anime_to_uid[anime_to_uid.animeid.==mal_uid, :uid])
    items, weights = get_neighborhood(uid + 1, W, 30)
    df = DataFrame(anime = items, weight = weights)
    get_series(df) |> display
end;

In [ ]:
ENV["DATAFRAMES_COLUMNS"] = 300;
ENV["DATAFRAMES_ROWS"] = 300;

In [ ]:
# good series to test: 721, 290, 189, 39535, 82, 12467

In [ ]:
display_related_series(721)